In [ ]:
library(ggplot2)
library(viridis)
library(ComplexHeatmap)

In [ ]:
exp = read.csv("../Acta2_exp/expression.csv.gz",row.names=1)

meta = read.csv("../Acta2_exp/All_sc_meta.csv.gz",row.names=1)

meta1 = meta[,c("celltype","Age_TimePostInjury","Age","TimePostInjury")]
data = cbind(exp,meta1)
df = data[data$celltype == "FB_Injury",]

cell_counts <- table(df$Age_TimePostInjury)
filtered_df <- df[df$Age_TimePostInjury %in% names(cell_counts[cell_counts >= 20]), ]

In [ ]:
library(dplyr)
result <- filtered_df %>%
  group_by(Age_TimePostInjury) %>%
  summarise(count = sum(Acta2 > 0),Mean = mean(Acta2),Sumcount = n())
result = as.data.frame(result)
result$prob = result$count /result$Sumcount 

In [ ]:
exp <- matrix(0, nrow = 1, ncol = length(result$Age_TimePostInjury)) 
colnames(exp) <- result$Age_TimePostInjury 
rownames(exp) <- "Acta2"
exp[1,]= result$Mean

In [ ]:
meta.size <- matrix(0, nrow = 1, ncol = length(result$Age_TimePostInjury)) 
colnames(meta.size) <- result$Age_TimePostInjury  
rownames(meta.size) <- "Acta2"
meta.size[1,]= result$prob

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
col_fun = circlize::colorRamp2(c(quantile(exp[1,], 0), quantile(exp[1,], 0.5), quantile(exp[1,], 0.75), quantile(exp[1,], 0.95), quantile(exp, 1)), viridis(9)[c(1,2,6,8,9)])

heatmap_legend_param = list(
    # title = grid::textGrob("proportion", gp = gpar(col = "white", fontsize = 5)),
    title_gp = gpar(fontsize = 10, col = "white"),
    labels_gp = gpar(fontsize = 10, col = "white")
)

cell_fun = function(j, i, x, y, w, h, fill){
          grid.rect(x = x, y = y, width = w, height = h,
                    gp = gpar(col = NA, fill = NA))
          grid.circle(x=x,y=y,r= meta.size[i, j] *1 * unit(20, "mm"),
                      gp = gpar(fill = col_fun(exp[i, j]), col = NA))}

dot.label = lapply(c(0,0.5,0.9,1), function(x){ quantile(meta.size[1,], x) %>% round(digits = 1) }) %>% unlist
lgd_list = list(
    Legend( labels = dot.label, title = "% Exp",
            graphics = list(
              function(x, y, w, h) grid.circle(x = x, y = y, r = 0.1 * unit(20, "mm"),
                                               gp = gpar(fill = "white")),
              function(x, y, w, h) grid.circle(x = x, y = y, r = 0.3 * unit(20, "mm"),
                                                gp = gpar(fill = "white")),
              function(x, y, w, h) grid.circle(x = x, y = y, r = 0.5 * unit(20, "mm"),
                                               gp = gpar(fill = "white")),
              # function(x, y, w, h) grid.circle(x = x, y = y, r = 3 * unit(1, "mm"),
              #                                  gp = gpar(fill = "white")),
              function(x, y, w, h) grid.circle(x = x, y = y, r = 0.6 * unit(20, "mm"),
                                               gp = gpar(fill = "white")))
            ))

In [ ]:
exp=as.data.frame(exp)
exp=exp[,c("E16.5_6H","E16.5_1D","E16.5_2D","E16.5_3D",
"E17.5_6H","E17.5_1D","E17.5_2D","E17.5_3D","E17.5_Born",
"E18.5_6H","E18.5_1D","E18.5_2D","E18.5_3D","E18.5_Born",
"P5_6H","P5_1D","P5_2D","P5_3D","P5_6D",
"Adult_1D","Adult_2D","Adult_5D","Adult_10D","Adult_19D")]

meta.size=as.data.frame(meta.size)
meta.size=meta.size[,c("E16.5_6H","E16.5_1D","E16.5_2D","E16.5_3D",
"E17.5_6H","E17.5_1D","E17.5_2D","E17.5_3D","E17.5_Born",
"E18.5_6H","E18.5_1D","E18.5_2D","E18.5_3D","E18.5_Born",
"P5_6H","P5_1D","P5_2D","P5_3D","P5_6D",
"Adult_1D","Adult_2D","Adult_5D","Adult_10D","Adult_19D")]


In [ ]:
options(repr.plot.width = 24, repr.plot.height = 4)
pdf('AR.exp_in_lineage.pdf', width = 24, height = 4)
hp <- ComplexHeatmap::Heatmap(exp, 
              name = "Ave exp",
              col = col_fun,
              rect_gp = gpar(type = "none"),
              cell_fun = cell_fun,
              #column_title_gp = gpar(fontsize = 10),
              #row_title_gp = gpar(fontsize = 10),
              #row_names_gp = gpar(fontsize = 8),
              #column_names_gp = gpar(fontsize = 8),
              show_column_names = TRUE,
              row_names_side = "left",
              cluster_rows = FALSE,
              cluster_columns = FALSE,
              border = "white",
              heatmap_legend_param = heatmap_legend_param)
#hp
draw(hp, annotation_legend_list = lgd_list)
dev.off()